In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

2025-04-01 07:07:00.293710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743491220.534824    7538 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743491220.598648    7538 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743491221.116025    7538 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743491221.116146    7538 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743491221.116150    7538 computation_placer.cc:177] computation placer alr

In [2]:
with open(r'/mnt/d/MSAIM/trimister-3 msaiml/NLP/NLP_labs/generative_text_model/data/poem.txt') as story:
  story_data = story.read()
import re                             
def clean_text(text):
  text = re.sub(r',', '', text)
  text = re.sub(r'\'', '',  text)
  text = re.sub(r'\"', '', text)
  text = re.sub(r'\(', '', text)
  text = re.sub(r'\)', '', text)
  text = re.sub(r'\n', '', text)
  text = re.sub(r'“', '', text)
  text = re.sub(r'”', '', text)
  text = re.sub(r'’', '', text)
  text = re.sub(r'\.', '', text)
  text = re.sub(r';', '', text)
  text = re.sub(r':', '', text)
  text = re.sub(r'\-', '', text)

  return text


In [3]:
lower_data = story_data.lower()           

split_data = lower_data.splitlines()

In [4]:
final = ''                               
for line in split_data:
  line = clean_text(line)
  final += '\n' + line

In [5]:
final_data = final.split('\n') 

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Instantiating the Tokenizer
max_vocab = 1000000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(final_data)

# Getting the total number of words of the data.
word2idx = tokenizer.word_index
print(len(word2idx))
print(word2idx)
vocab_size = len(word2idx) + 1        # Adding 1 to the vocab_size because the index starts from 1 not 0. This will make it uniform when using it further
print(vocab_size)

3078
{'the': 1, 'and': 2, 'to': 3, 'of': 4, 'a': 5, 'i': 6, 'in': 7, 'that': 8, 'it': 9, 'with': 10, 'is': 11, 'all': 12, 'as': 13, 'was': 14, 'for': 15, 'not': 16, 'my': 17, 'you': 18, 'be': 19, 'from': 20, 'he': 21, 'but': 22, 'on': 23, 'they': 24, 'have': 25, 'we': 26, 'or': 27, 'his': 28, 'by': 29, 'me': 30, 'are': 31, 'one': 32, 'at': 33, 'its': 34, 'when': 35, 'so': 36, 'this': 37, 'her': 38, 'them': 39, 'like': 40, 'she': 41, 'if': 42, 'our': 43, 'what': 44, 'where': 45, 'there': 46, 'no': 47, 'some': 48, 'which': 49, 'had': 50, 'their': 51, 'more': 52, 'will': 53, 'out': 54, 'then': 55, 'know': 56, 'now': 57, 'thy': 58, 'too': 59, 'see': 60, 'thou': 61, 'day': 62, 'yet': 63, 'would': 64, 'us': 65, 'who': 66, 'come': 67, 'nor': 68, 'could': 69, 'may': 70, 'been': 71, 'down': 72, 'up': 73, 'love': 74, 'away': 75, 'thought': 76, 'said': 77, 'an': 78, 'how': 79, 'here': 80, 'can': 81, 'such': 82, 'years': 83, 'though': 84, 'than': 85, 'do': 86, 'were': 87, 'go': 88, 'your': 89, 'le

In [7]:
input_seq = []

for line in final_data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_seq = token_list[:i+1]
    input_seq.append(n_gram_seq)

In [8]:
max_seq_length = max(len(x) for x in input_seq)
print(max_seq_length)

62


In [9]:
input_seq = np.array(pad_sequences(input_seq, maxlen=max_seq_length, padding='pre'))

In [10]:
xs = input_seq[:, :-1]        # xs contains every word in sentence except the last one because we are using this value to predict the y value
labels = input_seq[:, -1]     # labels contains only the last word of the sentence which will help in hot encoding the y value in next step
print("xs: ",xs)
print("labels:",labels)
from numpy import save
save('data.npy', labels)

xs:  [[   0    0    0 ...    0    0    6]
 [   0    0    0 ...    0    6   25]
 [   0    0    0 ...    6   25 1167]
 ...
 [   0    0    0 ...    2   77 3078]
 [   0    0    0 ...   77 3078   26]
 [   0    0    0 ... 3078   26   31]]
labels: [  25 1167    5 ...   26   31  349]


In [11]:
from tensorflow.keras.utils import to_categorical

ys = to_categorical(labels, num_classes=vocab_size)
print(ys)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

# using the functional APIs of keras to define the model

i = Input(shape=(max_seq_length - 1, ))                         
x = Embedding(vocab_size, 124)(i)
x = Dropout(0.2)(x)
x = LSTM(520, return_sequences=True)(x)
x = Bidirectional(layer=LSTM(340, return_sequences=True))(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(vocab_size, activation='softmax')(x)

model = Model(i,x)


I0000 00:00:1743491400.979091    7538 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [15]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()                                       # We can know about the shape of the model
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
r = model.fit(xs, ys, epochs=150)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 61)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 61, 124)        │       381,796 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 61, 124)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 61, 520)        │     1,341,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 61, 680)        │     2,341,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 680)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       697,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3079)           │     3,155,975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,918,635 (30.21 MB)

 Trainable params: 7,918,635 (30.21 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.8397 - loss: 0.6096
Epoch 2/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 22s 58ms/step - accuracy: 0.8483 - loss: 0.5813
Epoch 3/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.8420 - loss: 0.5883
Epoch 4/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.8447 - loss: 0.5799
Epoch 5/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 22s 58ms/step - accuracy: 0.8477 - loss: 0.5752
Epoch 6/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 21s 55ms/step - accuracy: 0.8410 - loss: 0.5932
Epoch 7/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - accuracy: 0.8475 - loss: 0.5640
Epoch 8/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.8548 - loss: 0.5437
Epoch 9/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 21s 56ms/step - accuracy: 0.8438 - loss: 0.5768
Epoch 10/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 22s 57ms/step - accuracy: 0.8367 - loss: 0.5764
Epoch 11/150
376/376 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - accuracy: 0.8438 - loss: 0.5580
Epoch 12/150
376/37

In [16]:
def predict_words(seed, no_words):
  for i in range(no_words):
    token_list = tokenizer.texts_to_sequences([seed])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=1)

    new_word = ''

    for word, index in tokenizer.word_index.items():
      if predicted == index:
        new_word = word
        break
    seed += " " + new_word
  print(seed)

In [21]:
seed_text = 'Love and Lust'
next_words = 30

predict_words(seed_text, next_words)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━

None
